<div><img style="float: left; padding-right: 3em;" src="https://avatars.githubusercontent.com/u/19476722" width="150" /><div/>

# Earth Data Science Coding Challenge!
Before we get started, make sure to read or review the guidelines below. These will help make sure that your code is **readable** and **reproducible**. 

## Don't get **caught** by these Jupyter notebook gotchas

<img src="https://miro.medium.com/v2/resize:fit:4800/format:webp/1*o0HleR7BSe8W-pTnmucqHA.jpeg" width=300 style="padding: 1em; border-style: solid; border-color: grey;" />

  > *Image source: https://alaskausfws.medium.com/whats-big-and-brown-and-loves-salmon-e1803579ee36*

These are the most common issues that will keep you from getting started and delay your code review:

1. When you try to run some code on GitHub Codespaces, you may be prompted to select a **kernel**.
   * The **kernel** refers to the version of Python you are using
   * You should use the **base** kernel, which should be the default option. 
   * You can also use the `Select Kernel` menu in the upper right to select the **base** kernel
2. Before you commit your work, make sure it runs **reproducibly** by clicking:
   1. `Restart` (this button won't appear until you've run some code), then
   2. `Run All`

## Check your code to make sure it's clean and easy to read

<img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSO1w9WrbwbuMLN14IezH-iq2HEGwO3JDvmo5Y_hQIy7k-Xo2gZH-mP2GUIG6RFWL04X1k&usqp=CAU" height=200 />

* Format all cells prior to submitting (right click on your code).
* Use expressive names for variables so you or the reader knows what they are. 
* Use comments to explain your code -- e.g. 
  ```python
  # This is a comment, it starts with a hash sign
  ```

## Label and describe your plots

![Source: https://xkcd.com/833](https://imgs.xkcd.com/comics/convincing.png)

Make sure each plot has:
  * A title that explains where and when the data are from
  * x- and y- axis labels with **units** where appropriate
  * A legend where appropriate


## Icons: how to use this notebook
We use the following icons to let you know when you need to change something to complete the challenge:
  * &#128187; means you need to write or edit some code.
  
  * &#128214;  indicates recommended reading
  
  * &#9998; marks written responses to questions
  
  * &#127798; is an optional extra challenge
  

---

# Land cover classification at the Mississppi Delta

In this notebook, you will use a k-means **unsupervised** clustering algorithm to group pixels by similar spectral signatures. k-means is an **exploratory** method for finding patterns in data. Because it is unsupervised, you don't need any training data for the model. You also can't tell how well it "performs" because the clusters will not correspond to any particular land cover class. However, we expect at least some of the clusters to be identifiable.

You will use the [harmonized Sentinal/Landsat multispectral dataset](https://lpdaac.usgs.gov/documents/1698/HLS_User_Guide_V2.pdf). You can access the data with an [Earthdata account](https://www.earthdata.nasa.gov/learn/get-started) by installing the [`earthaccess` library from NSIDC](https://github.com/nsidc/earthaccess):

```bash
conda install -c conda-forge -y earthaccess
```

## STEP 1: SET UP

YOUR TASK:
  1. Import all libraries you will need for this analysis
  2. Configure GDAL parameters to help avoid connection errors:
     ```python
     os.environ["GDAL_HTTP_MAX_RETRY"] = "5"
     os.environ["GDAL_HTTP_RETRY_DELAY"] = "1"
     ```

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

Below you can find code for a caching **decorator** which you can use in your code. To use the decorator:

```python
@cached(key, override)
def do_something(*args, **kwargs):
    ...
    return item_to_cache
```

This decorator will **pickle** the results of running the `do_something()` function, and only run the code if the results don't already exist. To override the caching, for example temporarily after making changes to your code, set `override=True`.

In [ ]:
def cached(key, override=False):
    """"""
    def compute_and_cache_decorator(compute_function):
        """"""
        def compute_and_cache(*args, **kwargs):
            """Perform a computation and cache, or load cached result"""
            filename = os.path.join(et.io.HOME, et.io.DATA_NAME, 'jars', f'{key}.pickle')
            
            # Check if the cache exists already or override caching
            if not os.path.exists(filename) or override:
                # Make jars directory if needed
                os.makedirs(os.path.dirname(filename), exist_ok=True)
                
                # Run the compute function as the user did
                result = compute_function(*args, **kwargs)
                
                # Pickle the object
                with open(filename, 'wb') as file:
                    pickle.dump(result, file)
            else:
                # Unpickle the object
                with open(filename, 'rb') as file:
                    result = pickle.load(file)
                    
            return result
        
        return compute_and_cache
    
    return compute_and_cache_decorator

## STEP 2: STUDY SITE

For this analysis, you will use a watershed from the [Water Boundary Dataset](https://www.usgs.gov/national-hydrography/access-national-hydrography-products), HU12 watersheds (WBDHU12.shp).

YOUR TASK:
  1. Download the Water Boundary Dataset for region 8 (Mississippi)
  2. Select watershed 080902030506
  3. Generate a site map of the watershed

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

I chose this watershed because it covers parts of New Orleans an is near the Mississippi Delta. Deltas are boundary areas between the land and the ocean, and tend to contain a rich variety of different land cover and land use types.

In the cell below, write a 2-3 sentence **site description** of this area that helps to put your analysis in context.

**YOUR SITE DESCRIPTION HERE**

![](https://media.baamboozle.com/uploads/images/510741/1651543763_75056_gif-url.gif)

## STEP 3: MULTISPECTRAL DATA

### Search for data

**YOUR TASK:**
  1. Log in to the `earthaccess` service using your Earthdata credentials:
     ```python
     earthaccess.login(persist=True)
     ```
  2. Modify the following sample code to search for granules of the HLSL30 product overlapping the watershed boundary from May to October 2023 (there should be 76 granules):
     ```python
     results = earthaccess.search_data(
         short_name="...",
         cloud_hosted=True,
         bounding_box=tuple(gdf.total_bounds),
         temporal=("...", "..."),
     )
     ```

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

### Compile information about each granule

I recommend building a GeoDataFrame, as this will allow you to plot the granules you are downloading and make sure they line up with your shapefile. You could also use a DataFrame, dictionary, or a custom object to store this information.

**YOUR TASK -- Write a function that:**
  1. For each search result:
      1. Get the following information (HINT: look at the ['umm'] values for each search result):
          - granule id (UR)
          - datetime
          - geometry (HINT: check out the shapely.geometry.Polygon class to convert points to a Polygon)
      2. Open the granule files. I recomment opening one granule at a time, e.g. with (`earthaccess.open([result]`).
      3. For each file (band), get the following information:
          - file handler returned from `earthaccess.open()`
          - tile id
          - band number
  2. Compile all the information you collected into a GeoDataFrame

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

### Open, crop, and mask data

This will be the most resource-intensive step. I recommend caching your results using the `cached` decorator or by writing your own caching code. I also recommend testing this step with one or two dates before running the full computation.

This code should include at least one **function** including a numpy-style docstring. A good place to start would be a function for opening a single masked raster, applying the appropriate scale parameter, and cropping.

**YOUR TASK:**
1. For each granule:
    1. Open the Fmask band, crop, and compute a quality mask for the granule. You can use the following code as a starting point, making sure that `mask_bits` contains the quality bits you want to consider:
         ```python
         # Expand into a new dimension of binary bits
         bits = np.unpackbits(da.astype(np.uint8)).reshape(da.shape + (-1,))

         # Select the required bits and check if any are flagged
         mask = np.prod(bits[..., mask_bits]==0, axis=-1)
         ```

    2. For each band that starts with 'B':
        1. Open the band, crop, and apply the scale factor
        2. Name the DataArray after the band using the `.name` attribute
        3. Apply the cloud mask using the `.where()` method
        4. Store the DataArray in your data structure (e.g. adding a GeoDataFrame column with the DataArray in it. Note that you will need to remove the rows for unused bands)

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

### Merge and Composite Data

You will notice for this watershed that:
1. The raster data for each date are spread across 4 granules
2. Any given image is incomplete because of clouds

**YOUR TASK:**
1. For each band:
    1. For each date:
        1. Merge all 4 granules
        2. Mask any negative values created by interpolating from the nodata value of -9999 (`rioxarray` should account for this, but doesn't appear to when merging. If you leave these values in they will create problems down the line)
    2. Concatenate the merged DataArrays along a new date dimension
    3. Take the mean in the date dimension to create a composite image that fills cloud gaps
    4. Add the band as a dimension, and give the DataArray a name
2. Concatenate along the band dimension

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

## STEP 4: K-MEANS

Cluster your data by spectral signature using the k-means algorithm. 

**YOUR TASK:**
1. Convert your DataArray into a **tidy** DataFrame of reflectance values (hint: check out the `.unstack()` method)
2. Filter out all rows with no data (all 0s or any N/A values)
3. Fit a k-means model. You can experiment with the number of groups to find what works best.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

## STEP 5: PLOT

**YOUR TASK:**
Create a plot that shows the k-means clusters next to an RGB image of the area. You may need to brighted your RGB image by multiplying it by 10.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

Don't forget to interpret your plot!

**YOUR PLOT HEADLINE AND DESCRIPTION HERE**

![](https://media.baamboozle.com/uploads/images/510741/1651543763_75056_gif-url.gif)